In [1]:
from sympy import *
from scipy.integrate import quad
x = symbols('x')

In [2]:
def jxbar_prime(pe, tb, jxbar, theta, alpha):
    return  pe ** (-alpha * theta) * (pe + tb) ** (-(1 - alpha) * theta) / (
            jxbar * pe ** (-alpha * theta) * (pe + tb) ** (-(1 - alpha) * theta) + (
            1 - jxbar) * (pe + (1- alpha) * tb) ** (-theta)) * jxbar

def j0bar_prime(pe,tb,jxbar, theta, alpha):
    return (pe + tb) ** (-(1 - alpha) * theta) / (
            jxbar * (pe + tb) ** (-(1 - alpha) * theta) + (1 - jxbar) * pe ** (
            -(1 - alpha) * theta)) * jxbar

def CeFF_prime(pe,tb,jxbar_prime, jxbar, sigmastar, theta, alpha):
    epsilonDstar = alpha + (1 - alpha) * sigmastar
    return Ceystar * ((1 - jxbar_prime) / (1 - jxbar)) ** (1 + (1 - sigmastar) / theta) * pe ** (-epsilonDstar)

def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1))

    Bfunvec1_prime = quad(tempFunction, 0, j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction, 0, jxbar_prime, args=(theta, sigmastar))[0]

    return (Bfunvec1_prime, Bfunvec2_prime)

In [3]:
Cex = 0.421601
Ceystar = 26.059874
Cey = 4.598402945
Cem = 1.196110986
jxbar = Cex/(Cex + Ceystar)
alpha = 0.85
theta = 4
## phi = 5
pe = 1.010698
tb = 1.762878
tau = 1.1
sigmastar = 1
sigma = 1
jxbar_p = jxbar_prime(pe, tb, jxbar, 4, 0.85)
j0bar_p = j0bar_prime(pe,tb,jxbar,theta,alpha)

In [4]:
def g(p, alpha = 0.15):
    return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha

def gprime(p, alpha = 0.15):
    return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))

In [43]:
def jxbar_new(pe, tb, jxbar, theta, alpha, Cex, Ceystar):
    num = g(pe + tb, alpha)**(-theta) * Cex
    denum = g(pe+tb, alpha)**(-theta) * Cex + (g(pe, alpha) + tb * gprime(pe, alpha))**(-theta) * Ceystar
    return num / denum

def j0bar_new(pe,tb,jxbar,theta,alpha,Cex,Ceystar):
    num = g(pe + tb, alpha)**(-theta) * Cex
    denum = g(pe+tb, alpha)**(-theta) * Cex + (g(pe, alpha))**(-theta) * Ceystar
    return num / denum

def Ceystar_new(pe,tb, jxbar_p, jxbar, sigmastar, theta, Ceystar, alpha):
    const = gprime(pe, alpha) * g(pe, alpha)**(-sigmastar) * Ceystar
    frac = ((1-jxbar_p)/ (1-jxbar))**(1+(1-sigmastar)/theta)
    return const * frac / (g(1,alpha)**(-sigmastar) * gprime(1,alpha))

def Cey_new(pe,tb, sigmastar, theta, Cey, alpha):
    const = gprime(pe + tb, alpha) * g(pe+tb, alpha)**(-sigma) * Cey
    return const / (g(1,alpha)**(-sigma) * gprime(1,alpha))

def Cex1_new(pe,tb, j0bar_p, j0bar, sigmastar, Cex, alpha):
    const = gprime(pe + tb, alpha) * g(pe+tb, alpha)**(-sigmastar) / (g(1,alpha)**(-sigmastar) * gprime(1,alpha)) * Cex
    frac = (j0bar_p / j0bar) ** (1+ (1-sigmastar)/theta)
    return const * frac

def Cex2_new(pe,tb,jxbar_p, j0bar_p, jxbar, sigmastar, theta):
    const = g(pe)**(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = ((1-jxbar)/jxbar)**(sigmastar/theta) * (theta + 1 - sigmastar)/theta
    jterm = 1/ jxbar**(1+(1-sigmastar)/theta)
    B1, B2 = imcomp_betas(j0bar_p, jxbar_p, theta, sigmastar)
    return const * frac * jterm * (B2- B1)
    

def Cem_new(pe, tb, sigma, Cem, alpha):
    const = gprime(pe + tb, alpha) * g(pe + tb, alpha)**(-sigma) / (g(1,alpha)**(-sigmastar) * gprime(1,alpha)) * Cem
    return const
    
def Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar, alpha):
    pterm = g(pe, alpha)**(1-sigmastar) *g(1)* Cex / (gprime(1,alpha) * g(1)**(1-sigmastar))
    num = (1-j0bar_p)**((theta + 1 - sigmastar)/theta) - (1-jxbar_p)**((theta + 1 - sigmastar) / theta)
    denum = jxbar * (1-jxbar)**((1-sigmastar)/theta)
    return pterm * num / denum

def Vgx1_new(pe, tb, j0_p, jxbar, Cex, theta, sigmastar):
    const = (pe + tb) * g(pe+tb) **(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1))
    jterm = (j0_p/ jxbar)**(1+(1-sigmastar)/theta)
    cterm = Cex / (g(1)**(-sigmastar) * gprime(1))
    return const * jterm * cterm

def S_new(pe, tb, Cex2_prime, Vgx2_prime):
    const = g(pe+tb) / gprime(pe+tb) * Cex2_prime - Vgx2_prime
    return const

def Vgm_new(pe, tb, Cem):
    return g(pe+tb)**(1-sigma) * Cem * g(1) / (g(1)**(1-sigma) * gprime(1))

In [26]:
jxbar_p

0.021877770950860884

In [40]:
Cex2_prime = Cex2_new(pe, tb, jxbar_p, j0bar_p, jxbar, sigmastar, theta)
Vgx2_prime = Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar, 1-alpha)
S_new(pe, tb, Cex2_prime, Vgx2_prime)

2.4222219185583245

In [41]:
Vgx2_prime

0.23174304261666928

In [20]:
Cex2_prime

0.14353121896650714

In [21]:
Vgx2_prime

2.3174304261666916

In [44]:
Vgm_new(pe,tb,Cem)

7.97407324